In [1]:
# !pip install transformers==4.18.0

In [2]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, TrainingArguments, Trainer
import tensorflow as tf
import numpy as np
import evaluate
import torch

c:\Program Files\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv("data/emails.csv")

df.rename(columns={'spam': 'labels'})

# data.head()
# df['spam'].value_counts()

,text,labels
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
...,...,...
5721,Subject: re : research and development charges...,0
5722,"Subject: re : receipts from visit jim , than...",0
5723,Subject: re : enron case study update wow ! a...,0
5724,"Subject: re : interest david , please , call...",0


In [4]:
train_data_df = df.sample(200, random_state=42)
eval_data_df = df.sample(200, random_state=45)
train_dataset = Dataset.from_pandas(train_data_df)
eval_dataset = Dataset.from_pandas(eval_data_df)
test_data_df = df.iloc[4000:4512]
test_dataset = Dataset.from_pandas(test_data_df)

In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# distilbert-base-uncased, bert-base-uncased

In [8]:
def tokenize_function(examples):
    tokenizer.encode(text='text', padding=True, truncation=True,max_length=512, add_special_tokens = True)
    return tokenizer(text='text', text_target='labels', padding="max_length", truncation=True)
    
tokenized_datasets_train = train_dataset.map(tokenize_function, batched=True)
tokenized_datasets_eval = eval_dataset.map(tokenize_function, batched=True)
tokenizer_datasets_test = test_dataset.map(tokenize_function, batched=True)

# examples['text']

Map:   0%|          | 0/200 [00:00<?, ? examples/s]


ArrowInvalid: Column 3 named input_ids expected length 200 but got length 512

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer")
TrainingArguments.label_names = ['labels']

In [ ]:
metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_eval,
    compute_metrics=compute_metrics,
    processing_class=tokenizer
)

In [ ]:
trainer.train()